In [81]:
import langchain

In [82]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [83]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [84]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [85]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000293AF41E620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000293AF41E770>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [86]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000293AF7C31C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000293AF7C17B0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [87]:
## create message
messages=[
    SystemMessage("you are a helpful AI assistant"),
    HumanMessage("What are the top 3 benifits of langchain?")
]

In [88]:
 ## invoke the model
response=model.invoke(messages)
response

AIMessage(content='Langchain is a platform that enables users to build and deploy AI models, particularly those that are multimodal (able to process and understand text, images, and other forms of data). Here are three potential benefits of Langchain:\n\n1. **Multimodal AI capabilities**: Langchain allows developers to create AI models that can process and understand multiple forms of data, such as text, images, and audio. This enables a wide range of applications, including image classification, object detection, natural language processing, and more. The ability to integrate multiple data types into a single model can lead to more accurate and informative results.\n\n2. **Improved AI Model Customization**: Langchain enables developers to fine-tune pre-trained AI models to specific use cases and domains, which can lead to more accurate and relevant results. This is achieved through a process called "prompt engineering," where developers can craft custom prompts to elicit specific resp

In [89]:
print(response.content)

Langchain is a platform that enables users to build and deploy AI models, particularly those that are multimodal (able to process and understand text, images, and other forms of data). Here are three potential benefits of Langchain:

1. **Multimodal AI capabilities**: Langchain allows developers to create AI models that can process and understand multiple forms of data, such as text, images, and audio. This enables a wide range of applications, including image classification, object detection, natural language processing, and more. The ability to integrate multiple data types into a single model can lead to more accurate and informative results.

2. **Improved AI Model Customization**: Langchain enables developers to fine-tune pre-trained AI models to specific use cases and domains, which can lead to more accurate and relevant results. This is achieved through a process called "prompt engineering," where developers can craft custom prompts to elicit specific responses from the AI model

In [90]:
model.invoke([HumanMessage("What is machine learning")])

AIMessage(content='Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time without being explicitly programmed.\n\nMachine learning works by feeding a machine learning model with a large dataset, and the model learns patterns, relationships, and trends within the data. This process allows the model to make predictions, classify objects, and make decisions based on the data it has learned.\n\nThere are several types of machine learning:\n\n1. **Supervised learning**: The model is trained on labeled data, where the correct output is already known. The model learns to make predictions based on the input data.\n2. **Unsupervised learning**: The model is trained on unlabeled data, and it must find patterns or relationships within the data on its own.\n3. **Reinforcement learning**: The model learns by interacting with an environment a

In [91]:
# streamin exemple
for chunk in model.stream(messages):
    print(chunk.content,end="",flush=True)

LangChain is an open-source framework for building large language models (LLMs) and their applications. Here are three of the top benefits of using LangChain:

1. **Easy Integration with LLMs**: LangChain provides a simple and seamless way to integrate with popular LLMs like LLaMA, Meta AI's Llama, and more. This makes it easier to build applications that leverage the power of LLMs, without requiring extensive knowledge of the underlying model architecture.

2. **Modular Architecture**: LangChain's modular design allows developers to break down complex tasks into smaller, manageable components. This makes it easier to build and maintain applications, as each component can be modified or replaced independently without affecting the rest of the system.

3. **Extensive Range of Tools and Functions**: LangChain includes a wide range of tools and functions for building and deploying LLM-based applications. These include support for tasks like data loading, model training, and deployment, as

# Dynamic Prompt Templates

In [92]:
from langchain.prompts import ChatPromptTemplate

## create translation app

translation_template=ChatPromptTemplate.from_messages([
    ("system","you are a professional translator.Translate the following {text} from {source_language} to {target_language}.Maintain the tone and style"),
    ("user","{text}")
])

## using template
prompt=translation_template.invoke({
    "source_language":"English",
    "target_language":"spanish",
    "text":"Langchain makes building AI application incredibility easy!"
})

In [93]:
prompt

ChatPromptValue(messages=[SystemMessage(content='you are a professional translator.Translate the following Langchain makes building AI application incredibility easy! from English to spanish.Maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building AI application incredibility easy!', additional_kwargs={}, response_metadata={})])

In [94]:
translated_response=model.invoke(prompt)
print(translated_response.content)

Langchain hace que construir aplicaciones de IA sea increíblemente fácil.


# building your first chain

In [102]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough ,RunnableLambda


def create_story_chain():
    ## template for story genration
    story_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "you are a creative storyteller. Write a short and engaging story based on a given theme, character, and setting"),
            ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
        ]
    )

    # template for story anylysis

    anylysis_prompt=ChatPromptTemplate.from_messages([
        ("system","you are a litrary critic. Anylyze the following story and provide insights."),
        ("user","{story}")
    ])

    # create a story function to pass a story to anylysis
    def analyze_story(story_text):
        return {"story":story_text}
    story_chain=(
        story_prompt| model | StrOutputParser
    )
    anylysis_chain=(
        story_chain
        | RunnableLambda(analyze_story)
        | anylysis_prompt
        | model
        | StrOutputParser()
    )

    return anylysis_chain

In [103]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are a creative storyteller. Write a short and engaging story based on a given theme, character, and setting'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000293AF41E620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000293AF41E770>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(StrOutputParser)
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['stor

In [104]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are a creative storyteller. Write a short and engaging story based on a given theme, character, and setting'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000293AF41E620>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000293AF41E770>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(StrOutputParser)
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['stor

In [105]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})


print("Story and Anylysis:")
print(result)

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given